In [25]:
import pandas as pd
import numpy as np
import time
import datetime
from collections import defaultdict
import gc

In [26]:
# 列表：店铺id，商品id，用户id，品牌，城市
# 1.计算每个列表的数量和销售等级总和
# 2.计算每个用户在各列表（商品、店铺、品牌）中所占比例

def dolastCount(data):
    
    keys = ['shop_id', 'item_id', 'user_id', 'item_brand_id', 'item_city_id', 'item_sales_level']

    for colname in keys:
        print(colname,'starting.....')
        count = data.groupby([colname]).apply(lambda x: x['instance_id'][(x['day']!=7).values].count()).reset_index(name='cnt_'+colname)
        sums = data.groupby([colname]).apply(lambda x: x['item_sales_level'][(x['day']!=7).values].sum()).reset_index(name='sum_'+colname)
        data = pd.merge(data,count, how='left', on=[colname])
        data = pd.merge(data,sums, how='left', on=[colname])
    
    for colname in keys:
        if(colname != 'user_id'):
            print('user_id_', colname, 'starting.....')
            count = data.groupby([colname, 'user_id']).apply(lambda x: x['instance_id'][(x['day']!=7).values].count()).reset_index(name='cnt_'+'user_id_'+colname)
            sums = data.groupby([colname, 'user_id']).apply(lambda x: x['item_sales_level'][(x['day']!=7).values].sum()).reset_index(name='sum_'+'user_id_'+colname)
            data = pd.merge(data,count, how='left', on=[colname, 'user_id'])
            data = pd.merge(data,sums, how='left', on=[colname, 'user_id'])
    
    data['item_user_ratio'] = data['cnt_user_id_item_id']/data['cnt_user_id']
    data['shop_user_ratio'] = data['cnt_user_id_shop_id']/data['cnt_user_id']
    data['brand_user_ratio'] = data['cnt_user_id_item_brand_id']/data['cnt_user_id'] 

    return data

In [27]:
# 1.计算每个小时内商品收藏量、商品浏览量和商品销售量的比率
# 2.判断每个商品的重复次数是否大于2
# 3.计算每个商店和每个商品类别在每天内的查询次数
# 4.计算每个商品在三天内的平均价格
# 5.计算每个商品在三天内的价格差

def doNew(data):

    # collected pv sales 之间的小时比率
    print('collected pv sales 之间的小时比率')
    coll_query = data.groupby(['day', 'hour'], as_index=False)['item_collected_level'].agg({'hour_query_collect': 'sum'})
    pv_query = data.groupby(['day', 'hour'], as_index=False)['item_pv_level'].agg({'hour_query_pv': 'sum'})
    sales_query = data.groupby(['day', 'hour'], as_index=False)['item_sales_level'].agg({'hour_query_sales': 'sum'})
    coll_query = coll_query.merge(pv_query, how='left', on=['day', 'hour'])
    coll_query = coll_query.merge(sales_query, how='left', on=['day', 'hour'])

    coll_query['coll_sales_hour_ratio'] = round(coll_query['hour_query_collect'] / coll_query['hour_query_sales'],5)
    coll_query['coll_sales_hour_ratio'] = coll_query['coll_sales_hour_ratio'] - coll_query['coll_sales_hour_ratio'].min()
    coll_query['pv_sales_hour_ratio'] = round(coll_query['hour_query_pv'] / coll_query['hour_query_sales'], 5)
    coll_query['pv_sales_hour_ratio'] = coll_query['pv_sales_hour_ratio'] - coll_query['pv_sales_hour_ratio'].min()
    
    del coll_query['hour_query_collect']
    del coll_query['hour_query_pv']
    del coll_query['hour_query_sales']

    data = pd.merge(data, coll_query, how='left', on=['day', 'hour'])
    
    #重复次数是否大于2
    print('重复次数是否大于2')
    subset = ['item_brand_id', 'item_id', 'shop_id', 'user_id']
    temp=data.groupby(subset)['is_trade'].count().reset_index()
    temp.columns=['item_brand_id', 'item_id', 'shop_id', 'user_id','large2']
    temp['large2']=1*(temp['large2']>2)
    data = pd.merge(data, temp, how='left', on=subset)
    
    #每天内的查询次数
    shop_query = data.groupby(['shop_id', 'day']).size().reset_index().rename(columns={0: 'shop_id_query_day'})
    category_2_query = data.groupby(['item_category_2', 'day']).size().reset_index().rename(columns={0: 'category_2_query_day'})
    data = pd.merge(data, shop_query, how='left', on=['shop_id', 'day'])
    data = pd.merge(data, category_2_query, how='left', on=['item_category_2', 'day'])

    #商品平均价格差
    print('price diff......')
    print('item_price_level')
    temp = data[['item_id', 'item_price_level']].loc[data.day==4]
    item_price_4 = temp.groupby(['item_id'], as_index=False)['item_price_level'].agg({'price_4': 'mean'})
    data = pd.merge(data, item_price_4, how='left', on='item_id')
    del temp
    gc.collect()
    print('item_price_level')
    temp = data[['item_id', 'item_price_level']].loc[data.day==5]
    item_price_5 = temp.groupby(['item_id'], as_index=False)['item_price_level'].agg({'price_5': 'mean'})
    data = pd.merge(data, item_price_5, how='left', on='item_id')
    del temp
    gc.collect()
    print('item_price_level')
    temp = data[['item_id', 'item_price_level']].loc[data.day==6]
    item_price_6 = temp.groupby(['item_id'], as_index=False)['item_price_level'].agg({'price_6': 'mean'})
    data = pd.merge(data, item_price_6, how='left', on='item_id')
    del temp
    gc.collect()

    data['price_diff_7_6'] = data['item_price_level'] = data['price_6']
    data['price_diff_7_5'] = data['item_price_level'] = data['price_5']
    data['price_diff_7_4'] = data['item_price_level'] = data['price_4']

    del data['price_6']
    del data['price_5']
    del data['price_4']

    return data

In [28]:
# 计算给定数据集中每个用户、商店、商品和商品品牌的历史出现次数。
# 遍历四个特征（user_id、shop_id、item_id、item_brand_id）
# 获取特征的值列表，使用字典存储该特征值出现的次数
# 遍历数据集中的每一行，将该行的特征值作为字典的键，将该键对应的值加1，并赋给名为till_now_cnt的numpy数组
# 将till_now_cnt数组添加到数据集中

def tillNow(data):
    
    for feat in ['user_id', 'shop_id', 'item_id', 'item_brand_id']:
        lists = data[feat].values
        dicts = defaultdict(lambda: 0)
        till_now_cnt = np.zeros(len(data))
        for i in range(len(data)):
            till_now_cnt[i] = dicts[lists[i]]
            dicts[lists[i]] += 1
        if(feat == 'item_brand_id'):
            data[feat.split('_')[1]+'_till_now_cnt'] = till_now_cnt
        else:
            data[feat.split('_')[0]+'_till_now_cnt'] = till_now_cnt

    return data

In [29]:
path = '/Users/apple/Desktop/data/'

   
train = pd.read_csv(path+'train_all.csv')
test = pd.read_csv(path+'test_all.csv')

data = pd.concat([train, test])

print('初始维度:', data.shape)

cols = data.columns.tolist()
keys = ['instance_id', 'day']
for k in keys:
    cols.remove(k)

初始维度: (99085, 33)


In [30]:
cols

['item_id',
 'item_brand_id',
 'item_city_id',
 'item_price_level',
 'item_sales_level',
 'item_collected_level',
 'item_pv_level',
 'user_id',
 'user_gender_id',
 'user_age_level',
 'user_occupation_id',
 'user_star_level',
 'context_id',
 'context_timestamp',
 'context_page_id',
 'shop_id',
 'shop_review_num_level',
 'shop_review_positive_rate',
 'shop_star_level',
 'shop_score_service',
 'shop_score_delivery',
 'shop_score_description',
 'is_trade',
 'time',
 'hour',
 'minute',
 'maphour',
 'mapmin',
 'item_category_0',
 'item_category_1',
 'item_category_2']

In [31]:
data = dolastCount(data)
print('dolastCount:', data.shape)

shop_id starting.....
item_id starting.....
user_id starting.....
item_brand_id starting.....
item_city_id starting.....
item_sales_level starting.....
user_id_ shop_id starting.....
user_id_ item_id starting.....
user_id_ item_brand_id starting.....
user_id_ item_city_id starting.....
user_id_ item_sales_level starting.....
dolastCount: (99085, 58)


In [32]:
data = doNew(data)
print('doNew:', data.shape)

collected pv sales 之间的小时比率
重复次数是否大于2
price diff......
item_price_level
item_price_level
item_price_level
doNew: (99085, 66)


In [33]:
data = tillNow(data)
print('tillNow:', data.shape) 

tillNow: (99085, 70)


In [34]:
data = data.drop(cols, axis=1)
path = '/Users/apple/Desktop/data/'

# 得到全部训练集
print('经过处理后,最终维度:', data.shape)
print(data.columns.tolist())
#data.to_csv(path+'401_list_till_feat.csv', index=False)
data.to_csv(path+'all_06.csv', index=False)

# 得到7号训练集
data7 = data.loc[data.day==7]
data7 = data7.drop('day', axis=1)
print('经过处理后,7号训练集最终维度:', data7.shape)
print(data7.columns.tolist())
#data7.to_csv(path+'401_list_till_feat7.csv', index=False)
data.to_csv(path+'day7_06.csv', index=False)

path = '/Users/apple/Desktop/data/'
# 将列和数据类型存储为 DataFrame
columns_data_types = pd.DataFrame({'Column': data.columns, 'Data Type':data.dtypes})

# 保存为 CSV 文件
columns_data_types.to_csv(path+'columns_data_types6.csv', index=False)

经过处理后,最终维度: (99085, 39)
['instance_id', 'day', 'cnt_shop_id', 'sum_shop_id', 'cnt_item_id', 'sum_item_id', 'cnt_user_id', 'sum_user_id', 'cnt_item_brand_id', 'sum_item_brand_id', 'cnt_item_city_id', 'sum_item_city_id', 'cnt_item_sales_level', 'sum_item_sales_level', 'cnt_user_id_shop_id', 'sum_user_id_shop_id', 'cnt_user_id_item_id', 'sum_user_id_item_id', 'cnt_user_id_item_brand_id', 'sum_user_id_item_brand_id', 'cnt_user_id_item_city_id', 'sum_user_id_item_city_id', 'cnt_user_id_item_sales_level', 'sum_user_id_item_sales_level', 'item_user_ratio', 'shop_user_ratio', 'brand_user_ratio', 'coll_sales_hour_ratio', 'pv_sales_hour_ratio', 'large2', 'shop_id_query_day', 'category_2_query_day', 'price_diff_7_6', 'price_diff_7_5', 'price_diff_7_4', 'user_till_now_cnt', 'shop_till_now_cnt', 'item_till_now_cnt', 'brand_till_now_cnt']
经过处理后,7号训练集最终维度: (10184, 38)
['instance_id', 'cnt_shop_id', 'sum_shop_id', 'cnt_item_id', 'sum_item_id', 'cnt_user_id', 'sum_user_id', 'cnt_item_brand_id', 'sum_it

In [35]:
data

,instance_id,day,cnt_shop_id,sum_shop_id,cnt_item_id,sum_item_id,cnt_user_id,sum_user_id,cnt_item_brand_id,sum_item_brand_id,...,large2,shop_id_query_day,category_2_query_day,price_diff_7_6,price_diff_7_5,price_diff_7_4,user_till_now_cnt,shop_till_now_cnt,item_till_now_cnt,brand_till_now_cnt
0,5.089880e+18,2,30,318,4,44,1,11,26,262,...,0,10,7448,6.0,NaN,NaN,0.0,0.0,0.0,0.0
1,2.163600e+18,5,232,2758,5,45,1,9,376,4176,...,0,46,7027,6.0,6.0,6.0,0.0,0.0,0.0,0.0
2,8.229730e+18,1,63,597,2,24,2,18,11583,113965,...,0,5,7661,NaN,NaN,NaN,0.0,0.0,0.0,0.0
3,8.978460e+18,2,3,8,1,4,1,4,5,6,...,0,1,7448,NaN,NaN,NaN,0.0,0.0,0.0,0.0
4,3.831630e+18,5,5,59,1,11,1,11,4,47,...,0,2,7027,NaN,5.0,NaN,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99080,9.047330e+18,3,2,14,2,14,1,7,2,14,...,0,1,7209,NaN,NaN,6.0,0.0,1.0,1.0,1.0
99081,1.156510e+18,6,27,274,9,90,1,10,27,274,...,0,4,11011,6.0,6.0,NaN,0.0,40.0,15.0,40.0
99082,9.093200e+18,2,36,392,20,240,1,12,11583,113965,...,0,3,7448,4.0,4.0,4.0,0.0,35.0,19.0,12813.0
99083,8.695670e+17,31,198,1858,2,14,2,16,198,1858,...,0,22,902,NaN,7.0,NaN,2.0,206.0,1.0,206.0


In [36]:
data7

,instance_id,cnt_shop_id,sum_shop_id,cnt_item_id,sum_item_id,cnt_user_id,sum_user_id,cnt_item_brand_id,sum_item_brand_id,cnt_item_city_id,...,large2,shop_id_query_day,category_2_query_day,price_diff_7_6,price_diff_7_5,price_diff_7_4,user_till_now_cnt,shop_till_now_cnt,item_till_now_cnt,brand_till_now_cnt
7,6.317860e+18,185,2002,17,187,0,0,151,1642,2256,...,0,9,303,7.0,7.0,7.0,0.0,0.0,0.0,0.0
28,4.999390e+18,163,2399,129,1935,0,0,170,2484,7341,...,0,24,6177,5.0,5.0,5.0,0.0,0.0,0.0,0.0
30,5.331950e+18,43,388,22,220,0,0,32,289,4508,...,0,5,6177,6.0,6.0,6.0,0.0,0.0,0.0,0.0
34,5.270040e+18,7,59,0,0,0,0,7,59,14624,...,0,1,6177,NaN,NaN,NaN,0.0,0.0,0.0,0.0
46,1.590950e+18,46,454,6,56,0,0,46,454,7341,...,0,10,6177,5.0,5.0,NaN,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99035,3.253290e+18,140,1318,77,693,0,0,352,3541,1364,...,0,23,259,6.0,6.0,6.0,0.0,162.0,91.0,413.0
99055,7.089270e+18,6,59,1,11,0,0,22,257,487,...,0,2,604,NaN,NaN,NaN,0.0,7.0,1.0,30.0
99060,4.900090e+18,768,8043,3,27,0,0,778,8127,14624,...,0,39,6177,NaN,NaN,NaN,0.0,806.0,3.0,818.0
99066,1.476060e+18,34,246,3,12,0,0,34,246,3350,...,0,5,657,8.0,8.0,NaN,0.0,38.0,6.0,38.0
